# Use python-pptx to generate powerpoint slides of data

In [ ]:
# Imports: 
import pathlib
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from tqdm.auto import tqdm

In [ ]:
# Define paths:
basePath = pathlib.Path('/Users/andrew/Library/CloudStorage/OneDrive-UCB-O365/research/data_analysis/rsoxs_suite/')

contrastPaths = basePath.joinpath('nexafs_plots/2023C3/material_optical_constants')
singlePath = contrastPaths.joinpath('single')
binaryPath = contrastPaths.joinpath('binary')

ISIsPath = basePath.joinpath('prsoxs_plots/2023C3/isi_carbon_v4')

outPath = pathlib.Path('/Users/andrew/Library/CloudStorage/OneDrive-UCB-O365/research/presentations/update_slides')

In [ ]:
selected_samples = [
    'PM6-Y6BO_CB',
    'PM6-Y6BO_CF',
    'PM6-Y6BO_CFCN',
    'PM6-Y6_CB',
    'PM6-Y6_CBCN',
    'PM6-Y6_CF',
    'PM6-Y6_CFCN',
    'PM6-Y7BO_CB',
    'PM6-Y7BO_CBCN',
    'PM6-Y7BO_CF',
    'PM6-Y7BO_CFCN',
    'PM6-Y7_CB',
    'PM6-Y7_CBCN',
    'PM6-Y7_CF',
    'PM6-Y7_CFCN',
    'PM6_CBCN',
    'PM6_CB_3000',
    'PM6_CFCN',
    'PM6_CF_3000',
    'Y6_CB_2000',
    'Y6_CB_2000_1',
    'Y6_CB_2500',
    'Y6_CB_3000',
    'Y6_CF_2000',
    'Y6_CF_2500'
]
selected_samples

In [ ]:
ISIs_all = set(ISIsPath.glob('*0.01-0.08*'))
# ISIs_sin = set(ISIsPath.glob('sin-sub*'))

ISIs = ISIs_all  # - ISIs_sin

ISIs = sorted(ISIs)
len([f.name for f in sorted(ISIs)])

In [ ]:
donor = 'PM6'
sorted([f.name for f in singlePath.glob(f'{donor}*282-290*6x4*')])
# sorted(singlePath.glob(f'{donor}_*282-290*6x4*'))

In [ ]:
donor = 'PM6'
# sorted([f.name for f in binaryPath.glob(f'{donor}-{acceptor}*282-290*')])
sorted(binaryPath.glob(f'{donor}-{acceptor}*282-290*'))

In [ ]:
# prs = Presentation()

# for i, layout in enumerate(prs.slide_layouts):
#     print(i, layout.name)

In [ ]:
titles = [
    'PM6-Y6BO',
    'PM6-Y6',
    'PM6-Y7BO',
    'PM6-Y7',
    'PM6',
    'Y6'
]

In [ ]:
# Generate slides:
prs = Presentation()

# Set slide dimensions for widescreen 16:9 aspect ratio
prs.slide_width = Inches(13.333)  # Width
prs.slide_height = Inches(7.5)    # Height

slide_layout = prs.slide_layouts[6]

for i, title in enumerate(titles):
    film_type = 'binary' if len(title.split('-')) == 2 else 'single'
    print(film_type)
    
    if film_type == 'binary':
        donor = title.split('-')[0]
        acceptor = title.split('-')[1]
        
        ISI_paths = [f for f in ISIs if (donor in f.name and acceptor+'_' in f.name)]
        # display(ISI_paths)
        
        # Select plots of each component
        donor_beta, donor_delta, donor_vac = sorted(singlePath.glob(f'{donor}_*282-290*6x4*'))
        acceptor_beta, acceptor_delta, acceptor_vac = sorted(singlePath.glob(f'{acceptor}_*282-290*6x4*'))
        
        # Select binary movie & frame
        binary_movie, binary_frame = sorted(binaryPath.glob(f'{donor}-{acceptor}_*282-290*'))

        # Add slide 1 (components)
        slide = prs.slides.add_slide(slide_layout)

        # Define image positions (adjust as necessary)
        # Donor plots (3)
        left_donor_delta = Inches(-0.13)
        left_donor_beta = Inches(4.4)
        left_donor_vac = Inches(8.73)
        top_donor_vac = top_donor_beta = top_donor_delta = Inches(1.29)
        height_donor_vac = height_donor_beta = height_donor_delta = Inches(3.23)
        width_donor_vac = width_donor_beta = width_donor_delta = Inches(4.85)

        # Acceptor plots (3)
        left_acceptor_delta = Inches(-0.13)
        left_acceptor_beta = Inches(4.4)
        left_acceptor_vac = Inches(8.73)
        top_acceptor_vac = top_acceptor_beta = top_acceptor_delta = Inches(4.27)
        height_acceptor_vac = height_acceptor_beta = height_acceptor_delta = Inches(3.23)
        width_acceptor_vac = width_acceptor_beta = width_acceptor_delta = Inches(4.85)
        
        # Add a new text box for the title instead of using the default title placeholder
        left_title = Inches(0)
        top_title = Inches(-0.01)
        width_title = Inches(13.33)
        height_title = Inches(0.65)
        textbox = slide.shapes.add_textbox(left_title, top_title, width_title, height_title)
        text_frame = textbox.text_frame
        text_frame.text = f'{title}'  
        for paragraph in text_frame.paragraphs:
            paragraph.alignment = PP_ALIGN.CENTER
            paragraph.font.size = Pt(36)
    
        # Add images to the slide
        slide.shapes.add_picture(str(donor_vac), left_donor_vac, top_donor_vac, height=height_donor_vac)
        slide.shapes.add_picture(str(donor_beta), left_donor_beta, top_donor_beta, height=height_donor_beta)
        slide.shapes.add_picture(str(donor_delta), left_donor_delta, top_donor_delta, height=height_donor_delta)
        
        slide.shapes.add_picture(str(acceptor_vac), left_acceptor_vac, top_acceptor_vac, height=height_acceptor_vac)
        slide.shapes.add_picture(str(acceptor_beta), left_acceptor_beta, top_acceptor_beta, height=height_acceptor_beta)
        slide.shapes.add_picture(str(acceptor_delta), left_acceptor_delta, top_acceptor_delta, height=height_acceptor_delta)
        
        # Add slide 2 (ISIs & binary):
        slide = prs.slides.add_slide(slide_layout)
        
        # ISI plots (4)
        left_ISI1 = Inches(-0.2)
        left_ISI2 = Inches(3.05)
        left_ISI3 = Inches(6.32)
        left_ISI4 = Inches(9.58)
        top_ISI4 = top_ISI3 = top_ISI2 = top_ISI1 = Inches(1.35)
        height_ISI4 = height_ISI3 = height_ISI2 = height_ISI1 = Inches(2.67)
        width_ISI4 = width_ISI3 = width_ISI2 = width_ISI1 = Inches(4)  
        
        # Contrasts (3)
        left_donor_vac = Inches(-0.74)
        left_acceptor_vac = Inches(3.84)
        left_binary_movie = Inches(8.42)
        top_binary_movie = top_donor_vac = top_acceptor_vac = Inches(3.93)        
        height_binary_movie = height_donor_vac = height_acceptor_vac = Inches(3.57)        
        width_binary_movie = width_donor_vac = width_acceptor_vac = Inches(5.35)        
        
        # Add a new text box for the title instead of using the default title placeholder
        left_title = Inches(0)
        top_title = Inches(-0.01)
        width_title = Inches(13.33)
        height_title = Inches(0.65)
        textbox = slide.shapes.add_textbox(left_title, top_title, width_title, height_title)
        text_frame = textbox.text_frame
        text_frame.text = f'{title}'

        for paragraph in text_frame.paragraphs:
            paragraph.alignment = PP_ALIGN.CENTER
            paragraph.font.size = Pt(36)
            
        # Add images to the slide
        try:
            slide.shapes.add_picture(str(ISI_paths[3]), left_ISI4, top_ISI4, height=height_ISI4, width=width_ISI4)  
        except IndexError:
            print('Missing a 4th ISI')
            pass
        slide.shapes.add_picture(str(ISI_paths[2]), left_ISI3, top_ISI3, height=height_ISI3, width=width_ISI3)
        slide.shapes.add_picture(str(ISI_paths[1]), left_ISI2, top_ISI2, height=height_ISI2, width=width_ISI2)
        slide.shapes.add_picture(str(ISI_paths[0]), left_ISI1, top_ISI1, height=height_ISI1, width=width_ISI1)

        slide.shapes.add_movie(str(binary_movie), left_binary_movie, top_binary_movie, height=height_binary_movie, 
                               width=width_binary_movie, poster_frame_image=str(binary_frame), mime_type='video/mp4')
        slide.shapes.add_picture(str(acceptor_vac), left_acceptor_vac, top_acceptor_vac, height=height_acceptor_vac)
        slide.shapes.add_picture(str(donor_vac), left_donor_vac, top_donor_vac, height=height_donor_vac)
        
    elif film_type == 'single':
        # ISIs
        ISI_paths = sorted(ISIsPath.glob(f'{title}_*'))
        if title == 'Y6':
            ISI_paths = ISI_paths[1:3]+ISI_paths[4:]
        
        # Constants:
        single_beta, single_delta, single_vac = sorted(singlePath.glob(f'{title}_*282-290*6x4*'))
        
        # Add slide 
        slide = prs.slides.add_slide(slide_layout)
        
        # Set positions:
        # ISI plots (4)
        left_ISI1 = Inches(-0.2)
        left_ISI2 = Inches(3.05)
        left_ISI3 = Inches(6.32)
        left_ISI4 = Inches(9.58)
        top_ISI4 = top_ISI3 = top_ISI2 = top_ISI1 = Inches(1.74)
        height_ISI4 = height_ISI3 = height_ISI2 = height_ISI1 = Inches(2.67)
        width_ISI4 = width_ISI3 = width_ISI2 = width_ISI1 = Inches(4)
        
        # Contrast plots (3)
        left_single_delta = Inches(-0.13)
        left_single_beta = Inches(4.4)
        left_single_vac = Inches(8.73)
        top_single_vac = top_single_beta = top_single_delta = Inches(4.27)
        height_single_vac = height_single_beta = height_single_delta = Inches(3.23)
        width_single_vac = width_single_beta = width_single_delta = Inches(4.85)    
        
        # Add a new text box for the title instead of using the default title placeholder
        left_title = Inches(0)
        top_title = Inches(-0.01)
        width_title = Inches(13.33)
        height_title = Inches(0.65)
        textbox = slide.shapes.add_textbox(left_title, top_title, width_title, height_title)
        text_frame = textbox.text_frame
        text_frame.text = f'{title}'

        for paragraph in text_frame.paragraphs:
            paragraph.alignment = PP_ALIGN.CENTER
            paragraph.font.size = Pt(36)
            
        # Add images to the slide
        slide.shapes.add_picture(str(ISI_paths[3]), left_ISI4, top_ISI4, height=height_ISI4, width=width_ISI4)  
        slide.shapes.add_picture(str(ISI_paths[2]), left_ISI3, top_ISI3, height=height_ISI3, width=width_ISI3)
        slide.shapes.add_picture(str(ISI_paths[1]), left_ISI2, top_ISI2, height=height_ISI2, width=width_ISI2)
        slide.shapes.add_picture(str(ISI_paths[0]), left_ISI1, top_ISI1, height=height_ISI1, width=width_ISI1)

        slide.shapes.add_picture(str(single_vac), left_single_vac, top_single_vac, height=height_single_vac)
        slide.shapes.add_picture(str(single_beta), left_single_beta, top_single_beta, height=height_single_beta)
        slide.shapes.add_picture(str(single_delta), left_single_delta, top_single_delta, height=height_single_delta)
                
        
prs.save(outPath.joinpath('2023C3_carbon_contrasts_v1.pptx'))
print('done!')

In [ ]:
# Generate slides:

prs = Presentation()

# Set slide dimensions for widescreen 16:9 aspect ratio
prs.slide_width = Inches(13.333)  # Width
prs.slide_height = Inches(7.5)    # Height

slide_layout = prs.slide_layouts[6]

for i, movie_pair in enumerate(tqdm(detector_movies_zipped)):
    # Select appropriate path for each file
    movie1Path = movie_pair[0] 
    movie2Path = movie_pair[1]     

    frame1Path = detector_frames_zipped[i][0] 
    frame2Path = detector_frames_zipped[i][1] 
    
    IvQMapPath = IvQ_maps[i]
    IvQSelectPath = IvQ_selects[i]   
    
    AvQMapPath = AvQ_maps[i]
    AvQSelectPath = AvQ_selects[i]

    ISIPath = ISIs[i]
    
    fname_list = movie1Path.name.split('_')    
    sample_name_list = fname_list[0:3]
    sample_name = '_'.join(sample_name_list)    
    
    
    # Add slide 
    slide = prs.slides.add_slide(slide_layout)
    
    # Define image positions (these are just placeholders, adjust as necessary)
    left_movie1 = Inches(-0.11)
    top_movie1 = Inches(0.43)
    height_movie1 = Inches(3.76)
    width_movie1 = Inches(4.6)
    
    left_movie2 = Inches(-0.11)
    top_movie2 = Inches(3.83)
    height_movie2 = Inches(3.76)
    width_movie2 = Inches(4.6)
    
    left_IvQMap = Inches(4.92)
    top_IvQMap = Inches(3.81)
    height_IvQMap = Inches(3.75)

    left_AvQMap = Inches(8.67)
    top_AvQMap = Inches(0.69)
    height_AvQMap = Inches(3.14)    

    left_ISI = Inches(4.49)
    top_ISI = Inches(0.64)
    height_ISI = Inches(3.19)
    
    # Add images to the slide
    slide.shapes.add_movie(str(movie2Path), left_movie2, top_movie2, height=height_movie2, width=width_movie2, poster_frame_image=str(frame2Path), mime_type='video/mp4')
    slide.shapes.add_movie(str(movie1Path), left_movie1, top_movie1, height=height_movie1, width=width_movie1, poster_frame_image=str(frame1Path), mime_type='video/mp4')
    
    slide.shapes.add_picture(str(IvQMapPath), left_IvQMap, top_IvQMap, height=height_IvQMap)
    slide.shapes.add_picture(str(AvQMapPath), left_AvQMap, top_AvQMap, height=height_AvQMap)
    slide.shapes.add_picture(str(ISIPath), left_ISI, top_ISI, height=height_ISI)
    
    # # Format title
    # title = slide.shapes.title
    # title.left = Inches(0)
    # title.top = Inches(-0.01)
    # title.width = Inches(13.33)
    # title.height = Inches(0.65)
    # title.text = f'Sample: {sample_name}'
    # for paragraph in title.text_frame.paragraphs:
    #     paragraph.alignment = PP_ALIGN.CENTER
    #     paragraph.font.size = Pt(36)
        
    # Add a new text box for the title instead of using the default title placeholder
    left_title = Inches(0)
    top_title = Inches(-0.01)
    width_title = Inches(13.33)
    height_title = Inches(0.65)
    textbox = slide.shapes.add_textbox(left_title, top_title, width_title, height_title)
    text_frame = textbox.text_frame
    text_frame.text = f'Sample: {sample_name}'

    for paragraph in text_frame.paragraphs:
        paragraph.alignment = PP_ALIGN.CENTER
        paragraph.font.size = Pt(36)


    # Add slide 
    slide = prs.slides.add_slide(slide_layout)
    
    # Define image positions (these are just placeholders, adjust as necessary)
    left_IvQSelect = Inches(-0.02)
    top_IvQSelect = Inches(2.18)
    height_IvQSelect = Inches(5.17)

    left_AvQSelect = Inches(7.01)
    top_AvQSelect = Inches(2.81)
    height_AvQSelect = Inches(4.21)  
    
    slide.shapes.add_picture(str(IvQSelectPath), left_IvQSelect, top_IvQSelect, height=height_IvQSelect)
    slide.shapes.add_picture(str(AvQSelectPath), left_AvQSelect, top_AvQSelect, height=height_AvQSelect)
            
    # Add a new text box for the title instead of using the default title placeholder
    left_title = Inches(0)
    top_title = Inches(-0.01)
    width_title = Inches(13.33)
    height_title = Inches(0.65)
    textbox = slide.shapes.add_textbox(left_title, top_title, width_title, height_title)
    text_frame = textbox.text_frame
    text_frame.text = f'Sample: {sample_name}'

    for paragraph in text_frame.paragraphs:
        paragraph.alignment = PP_ALIGN.CENTER
        paragraph.font.size = Pt(36)
        
        
prs.save(outPath.joinpath('2023C3_nitrogen_prsoxs_auto-summary_udpated_v1.pptx'))
print('done!')

In [ ]:
for moviePath in tqdm(sorted(moviePaths.glob('*mp4'))):
    # Select appropriate path for each file
    poster_frame_path = moviePaths.joinpath(f'{moviePath.stem}.jpeg')
    fname_list = moviePath.name.split('_')
    pol_str = fname_list[-1][:-4]

    
#     if pol_str == 'pol0deg':
#         pol_orientation = 'Horizontal'
#     elif pol_str == 'pol90deg':
#         pol_orientation = 'Vertical'
        
    try:
        if len(fname_list)==4:
            sample_name = fname_list[0]
            # print(sample_name)
            linecutMapPath = list(linecutMapsPath.glob(f'{sample_name}_*pol0deg*q0.01-0.08*'))[0]
            arMapPath = list(arMapsPath.glob(f'vset*{sample_name}_*pol0deg*q-0.01-0.08*'))[0]
            ISIPath = list(ISIsPath.glob(f'{sample_name}_*qrange-0.01-0.08*pol0deg*'))[0]
        elif len(fname_list)==5:
            sample_name_list = fname_list[0:2]
            joined_sample_name = '_'.join(sample_name_list)     
            # print(joined_sample_name)
            linecutMapPath = list(linecutMapsPath.glob(f'{joined_sample_name}_*pol0deg*q0.01-0.08*'))[0]
            arMapPath = list(arMapsPath.glob(f'vset*{joined_sample_name}_*pol0deg*q-0.01-0.08*'))[0]
            ISIPath = list(ISIsPath.glob(f'{joined_sample_name}_*qrange-0.01-0.08*pol0deg*'))[0]
            
        print(linecutMapPath)
        print(arMapPath)
        print(ISIPath)
    except IndexError:
        pass
        
    # elif len(fname_list) == 7:
    #     sample_name_list = fname_list[0:4]
    #     joined_sample_name = '_'.join(sample_name_list)
    #     linecutMapPath = list(linecutMapsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]
    #     arMapPath = list(arMapsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]
    #     ISIPath = list(ISIsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]
    # elif len(fname_list) == 4:
    #     joined_sample_name = fname_list[0]
    #     linecutMapPath = list(linecutMapsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]
    #     arMapPath = list(arMapsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]
    #     ISIPath = list(ISIsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]

In [ ]:
[f.name for f in ISIsPath.glob(f'{sample_name}*')]

In [ ]:
sample_name

In [ ]:
# Generate slides:

prs = Presentation()

# Set slide dimensions for widescreen 16:9 aspect ratio
prs.slide_width = Inches(13.333)  # Width
prs.slide_height = Inches(7.5)    # Height

slide_layout = prs.slide_layouts[5]

for moviePath in tqdm(sorted(moviePaths.glob('*mp4'))):
    # Select appropriate path for each file
    poster_frame_path = moviePaths.joinpath(f'{moviePath.stem}.jpeg')
    fname_list = moviePath.name.split('_')
    pol_str = fname_list[-1][:-4]
    

    if pol_str == 'pol90deg':
        pol_orientation = 'Vertical'
    else:
        pol_orientation = 'Horizontal'
        
    try:
        if len(fname_list)==4:
            sample_name = fname_list[0]
            print(sample_name)
            linecutMapPath = list(linecutMapsPath.glob(f'{sample_name}_c*polavgdeg*q0.01-0.08*'))[0]
            arMapPath = list(arMapsPath.glob(f'vset*{sample_name}_c*polavgdeg*q-0.01-0.08*'))[0]
            ISIPath = list(ISIsPath.glob(f'{sample_name}_c*qrange-0.01-0.08*polavgdeg*'))[0]
        elif len(fname_list)==5:
            sample_name_list = fname_list[0:2]
            sample_name = '_'.join(sample_name_list)     
            print(sample_name)
            linecutMapPath = list(linecutMapsPath.glob(f'{sample_name}_*polavgdeg*q0.01-0.08*'))[0]
            arMapPath = list(arMapsPath.glob(f'vset*{sample_name}_*polavgdeg*q-0.01-0.08*'))[0]
            ISIPath = list(ISIsPath.glob(f'{sample_name}_*qrange-0.01-0.08*polavgdeg*'))[0]
            
        print(linecutMapPath)
        print(arMapPath)
        print(ISIPath)
        if pol_orientation == 'Horizontal':
            # Add slide 
            slide = prs.slides.add_slide(slide_layout)

            # Formate title
            title = slide.shapes.title
            title.left = Inches(0.22)
            title.top = Inches(0.4)
            title.width = Inches(4.8)
            title.height = Inches(2.91)
            title.text = f'Sample:\n{sample_name}\n\n{pol_orientation} X-ray Polarization'
            for paragraph in title.text_frame.paragraphs:
                paragraph.alignment = PP_ALIGN.LEFT
                paragraph.font.size = Pt(36)

            # Define image positions (these are just placeholders, adjust as necessary)
            left_movie = Inches(0)
            top_movie = Inches(3.41)
            height_movie = Inches(4.09)
            width_movie = Inches(5)

            left_linecutMap = Inches(5.08)
            top_linecutMap = Inches(3.5)
            height_linecutMap = Inches(3.75)

            left_arMap = Inches(5)
            top_arMap = Inches(0.25)
            height_arMap = Inches(3)    

            left_ISI = Inches(9.45)
            top_ISI = Inches(0.25)
            height_ISI = Inches(3)

            # Add images to the slide
            slide.shapes.add_movie(str(moviePath), left_movie, top_movie, height=height_movie, width=width_movie, poster_frame_image=str(poster_frame_path), mime_type='video/mp4')
            slide.shapes.add_picture(str(linecutMapPath), left_linecutMap, top_linecutMap, height=height_linecutMap)
            slide.shapes.add_picture(str(arMapPath), left_arMap, top_arMap, height=height_arMap)
            slide.shapes.add_picture(str(ISIPath), left_ISI, top_ISI, height=height_ISI)
            
    except IndexError:
        print('Only movie, skipping this slide')
        
        
prs.save(outPath.joinpath('2022C2_carbon_saxs_prsoxs_auto-summary_horizontal_v2.pptx'))